<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/Hybrid%20LR-Bi-LSTM%20for%204%20class%20classifications%20based%20on%20WSNBFSF%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hybrid LR-Bi-LSTM for 4 class classifications based on WSNBFSF dataset**

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Flatten

In [2]:
# Load dataset
dataset = pd.read_csv("dataset.csv")
dataset = dataset.dropna() # Remove missing values
X = dataset.drop(['Class'], axis=1)
y = dataset['Class']

In [3]:
dataset.isnull().sum()

Event             0
Time              0
S_Node            0
Node_id           0
Rest_Energy       0
Trace_Level       0
Mac_Type_Pckt     0
Source_IP_Port    0
Des_IP_Port       0
Packet_Size       0
TTL               0
Hop_Count         0
Broadcast_ID      0
Dest_Node_Num     0
Dest_Seq_Num      0
Src_Node_ID       0
Src_Seq_Num       0
Class             0
dtype: int64

### **Data Preprocessing**

In [4]:
# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(dataset['Class'])

In [5]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(dataset.drop(['Class'], axis=1))

In [6]:
# Data preprocessing
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Hybrid combination of LR-Bi-LSTM for 4 class classifications**

In [8]:
# Train logistic regression model
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print("Logistic Regression Accuracy:", lr_acc)
print(classification_report(y_test, lr_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.7895435155815314
              precision    recall  f1-score   support

           0       0.77      0.56      0.65     52705
           1       0.71      0.73      0.72     52517
           2       0.83      0.87      0.85     52703
           3       0.84      0.99      0.91     52356

    accuracy                           0.79    210281
   macro avg       0.79      0.79      0.78    210281
weighted avg       0.79      0.79      0.78    210281



In [9]:
# Train Bi-LSTM model
num_classes = len(np.unique(y))
input_dim = X_train.shape[1]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
lstm_model = Sequential()
lstm_model.add(Bidirectional(LSTM(64, input_shape=(1, input_dim), activation='relu', return_sequences=True)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(32, activation='relu'))
lstm_model.add(Dense(4, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
y_train_onehot = pd.get_dummies(y_train).values
lstm_model.fit(X_train, y_train_onehot, epochs=15, batch_size=32, verbose=0)
lstm_pred = lstm_model.predict(X_test)
lstm_pred = np.argmax(lstm_pred, axis=1)
lstm_acc = accuracy_score(y_test, lstm_pred)
print("Bi-LSTM Accuracy:", lstm_acc)
print(classification_report(y_test, lstm_pred))

6572/6572 [==============================] - 20s 3ms/step
Bi-LSTM Accuracy: 0.9746006534113876
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     52705
           1       0.95      0.98      0.96     52517
           2       0.98      1.00      0.99     52703
           3       0.99      0.97      0.98     52356

    accuracy                           0.97    210281
   macro avg       0.98      0.97      0.97    210281
weighted avg       0.98      0.97      0.97    210281



In [11]:
# Combine predictions
lr_probs = lr_model.predict_proba(np.reshape(X_test, (X_test.shape[0], -1)))
lstm_probs = lstm_model.predict(X_test)
combined_probs = (lr_probs + lstm_probs) / 2
combined_pred = np.argmax(combined_probs, axis=1)
combined_acc = accuracy_score(y_test, combined_pred)
print("Combined Accuracy:", combined_acc)
print(classification_report(y_test, combined_pred))

6572/6572 [==============================] - 21s 3ms/step
Combined Accuracy: 0.9648850823421993
              precision    recall  f1-score   support

           0       0.99      0.94      0.96     52705
           1       0.95      0.94      0.95     52517
           2       0.98      0.99      0.98     52703
           3       0.94      0.99      0.96     52356

    accuracy                           0.96    210281
   macro avg       0.97      0.96      0.96    210281
weighted avg       0.97      0.96      0.96    210281

